In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_91'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
# tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def read_passages(path_data,test_size=0.1):
    df = pd.read_csv(path_data)

    documents = df['processed'].to_list()
    labels_str = df['target'].to_list()
    
    samples = documents
    
    labels_list = sorted(list(set(labels_str)))
    labels_all = {l:idx for idx, l in enumerate(labels_list)}
    labels = [labels_all[label_str] for label_str in labels_str]

    return train_test_split(samples, labels, test_size=test_size), labels_list
# end

In [5]:
# call the function
dir_data = 'datasource'
name_data_file = 'goscv_555_202205240000.csv'
path_data_relative = os.path.join(dir_data, name_data_file)
(train_samples, valid_samples, train_labels, valid_labels), target_names = read_passages(path_data_relative)

In [6]:
# train_samples[837]

In [7]:
train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [8]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = SimpleDataset(train_encodings, train_labels)
valid_dataset = SimpleDataset(valid_encodings, valid_labels)

In [10]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
model = DistilBertForSequenceClassification.from_pretrained(model_info, num_labels=len(target_names))

load model from official


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    # print('labels: {}'.format(labels))
    # print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=25,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=10,
    metric_for_best_model='f1'
)

In [13]:
# trainer = Trainer(
#     model=model,                         # the instantiated Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     compute_metrics=compute_metrics,     # the callback that computes metrics of interest
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
# )

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [14]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.108600,0.732795,0.785714,0.916190,0.503532,0.487906
2,0.009000,0.665781,0.875000,0.945959,0.687275,0.669222
3,0.108900,0.535623,0.892857,0.927444,0.826761,0.803328
4,0.000900,0.661266,0.875000,0.917857,0.818358,0.794195
5,0.001600,0.611147,0.910714,0.935043,0.837750,0.813110
6,0.007300,0.627615,0.910714,0.935043,0.837750,0.813110
7,0.001200,0.519851,0.910714,0.954347,0.876966,0.891252
8,0.000400,0.598014,0.892857,0.947344,0.829347,0.816222
9,0.000800,0.608599,0.910714,0.954347,0.876966,0.891252
10,0.000400,0.642215,0.910714,0.954347,0.876966,0.891252


<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to

TrainOutput(global_step=5250, training_loss=0.12019535274127806, metrics={'train_runtime': 505.8672, 'train_samples_per_second': 24.661, 'train_steps_per_second': 12.355, 'total_flos': 1388249646787584.0, 'train_loss': 0.12019535274127806, 'epoch': 21.0})

In [15]:
trainer.evaluate()

<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.5917195081710815,
 'eval_accuracy': 0.9285714285714286,
 'eval_precision': 0.9622252747252747,
 'eval_recall': 0.9245852187028658,
 'eval_f1': 0.937928737928738,
 'eval_runtime': 0.2368,
 'eval_samples_per_second': 236.464,
 'eval_steps_per_second': 118.232,
 'epoch': 21.0}

In [57]:
from torch import BoolTensor
device = "cuda"

input_sample = train_encodings['input_ids'][0]
input_sample = input_sample.to(device)

masks_sample = train_encodings['attention_mask'][0]
indicates_sample = BoolTensor(masks_sample == 1)

masks_sample = masks_sample.to(device)
indicates_sample = indicates_sample.to(device)

In [58]:
result = model(input_sample, attention_mask=masks_sample ,output_attentions=True, output_hidden_states=True)

In [65]:
result.attentions[-1].cpu()

tensor([[[[0.0588, 0.0147, 0.0138,  ..., 0.0000, 0.0000, 0.0000],
          [0.0268, 0.0443, 0.0263,  ..., 0.0000, 0.0000, 0.0000],
          [0.0322, 0.0490, 0.0400,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.1874, 0.0056, 0.0048,  ..., 0.0000, 0.0000, 0.0000],
          [0.1982, 0.0039, 0.0030,  ..., 0.0000, 0.0000, 0.0000],
          [0.1999, 0.0067, 0.0056,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.1005, 0.0225, 0.0047,  ..., 0.0000, 0.0000, 0.0000],
          [0.0212, 0.0346, 0.0712,  ..., 0.0000, 0.0000, 0.0000],
          [0.0148, 0.0688, 0.0816,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0994, 0.0024, 0.0010,  ..., 0.0000, 0.0000, 0.0000],
          [0.0941, 0.0013, 0.0007,  ..., 0.0000, 0.0000, 0.0000],
          [0.0834, 0.0032, 0.0015,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0346, 0.0294, 0.0153,  ..., 0.0000, 0.0000, 0.0000],
          [0.0165, 0.0557, 0.0375,  ..., 0.0000, 0.0000, 0.0000],
          [0.0104, 0.0536, 0.0466,  ..., 0

In [50]:
torch.masked_select((torch.sum(result['attentions'][-1][:,:,0,:], 1) / result['attentions'][-1].shape[1])[0], indicates_sample).tolist()[1:-1]

[0.02458353340625763,
 0.015769971534609795,
 0.01693333126604557,
 0.015847831964492798,
 0.017280802130699158,
 0.020594904199242592,
 0.01508481428027153,
 0.020402587950229645,
 0.02642880007624626,
 0.014142136089503765,
 0.02215564250946045,
 0.019940588623285294,
 0.016019348055124283,
 0.0162869393825531,
 0.01800132915377617,
 0.05113176256418228,
 0.03984392061829567,
 0.02846301719546318,
 0.03237047791481018,
 0.037791114300489426,
 0.019019294530153275,
 0.02857819013297558,
 0.045432932674884796,
 0.01855025440454483,
 0.01677907630801201,
 0.02137036621570587,
 0.013771604746580124,
 0.020632941275835037,
 0.018100213259458542,
 0.05540812015533447,
 0.027427103370428085,
 0.02120598778128624]

In [100]:
from torch import IntTensor
DEVICE_DEFAULT = 'cuda'
def predict_plus(input_tokenized, model):
    masks_sample = input_tokenized.attention_mask

    indicates_sample = BoolTensor(masks_sample == 1)
    indicates_sample = indicates_sample.to(DEVICE_DEFAULT)

    out = model(**input_tokenized.to(DEVICE_DEFAULT), output_attentions=True)

    logits = out.logits.cpu().tolist()
    attentions = out.attentions[-1].cpu()

    attentions_sum = torch.masked_select((torch.sum(attentions[:, :, 0, :], 1) / attentions.shape[1])[0],
                        indicates_sample).tolist()[1:-1]

    return logits, attentions_sum
# end

In [101]:
test1_encoding = tokenizer.batch_encode_plus([train_samples[0]], truncation=True, padding=True, max_length=max_length, return_tensors='pt')
predict_plus(test1_encoding, model)

([[-2.268066644668579,
   -3.006159543991089,
   -1.8981214761734009,
   8.328487396240234,
   -2.4117915630340576,
   -2.0045924186706543,
   -2.5799660682678223]],
 [0.02458353340625763,
  0.01576998271048069,
  0.016933344304561615,
  0.015847833827137947,
  0.017280809581279755,
  0.02059491164982319,
  0.01508481428027153,
  0.0204025711864233,
  0.026428798213601112,
  0.014142129570245743,
  0.022155655547976494,
  0.019940590485930443,
  0.01601935364305973,
  0.0162869431078434,
  0.018001336604356766,
  0.051131755113601685,
  0.03984392061829567,
  0.028463007882237434,
  0.03237047791481018,
  0.03779113292694092,
  0.019019296392798424,
  0.028578199446201324,
  0.045432936400175095,
  0.018550259992480278,
  0.016779057681560516,
  0.021370351314544678,
  0.013771605677902699,
  0.020632950589060783,
  0.018100207671523094,
  0.05540815368294716,
  0.027427084743976593,
  0.021206000819802284])

In [18]:
def get_attention_1(input_sample, result_model):
    scores_attention = (torch.sum(result['attentions'][-1][:,:,0,:], 1) / result['attentions'][-1].shape[1])[0].tolist()
    ids_input = input_sample.tolist()
    return list(zip(ids_input, scores_attention))
# end

def get_attention_2(encoder, model, index_input=0):
    
    device = "cuda:0"
    
    input_sample = train_encodings['input_ids'][index_input]
    input_sample = input_sample.to(device)
    
    masks_sample = train_encodings['attention_mask'][index_input]
    masks_sample = masks_sample.to(device)
    
    result = model(input_sample, attention_mask=masks_sample,output_attentions=True, output_hidden_states=True)
    
    scores_attention = (torch.sum(result['attentions'][-1][:,:,0,:], 1) / result['attentions'][-1].shape[1])[0].tolist()
    ids_input = input_sample.tolist()
    return list(zip(ids_input, scores_attention))
# end
    
    

In [19]:
# [(a, float('{:.4f}'.format(b))) for a,b in get_attention_1(input_sample, result)[:37]]
[(a, float('{:.4f}'.format(b))) for a,b in get_attention_2(train_encodings, model, 3)[:100]]

In [29]:
model(input_sample).logits

tensor([[-0.6371,  1.4537, -2.4508, -0.1007, -1.6882, -0.8329, -1.7313]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [30]:
model(input_sample, masks_sample).logits

tensor([[-2.2681, -3.0062, -1.8981,  8.3285, -2.4118, -2.0046, -2.5800]],
       device='cuda:0', grad_fn=<AddmmBackward>)